In [1]:
import sys
# Installation forcée des dépendances pour éviter les erreurs de modules
!{sys.executable} -m pip install langchain-ollama langchain-community langchain-core faiss-cpu codecarbon

  Using cached codecarbon-3.2.1-py3-none-any.whl.metadata (11 kB)
  Using cached arrow-1.4.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached fief_client-0.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached prometheus_client-0.24.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached nvidia_ml_py-13.590.48-py3-none-any.whl.metadata (9.8 kB)
  Using cached questionary-2.1.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached rich-14.3.1-py3-none-any.whl.metadata (18 kB)
  Using cached typer-0.21.1-py3-none-any.whl.metadata (16 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jwcrypto-1.5.6-py3-none-any.whl.metadata (3.1 kB)
  Using cached yaspin-3.4.0-py3-none-any.whl.metadata (15 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using ca


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_core.messages import SystemMessage, HumanMessage
from codecarbon import EmissionsTracker
import os
import time
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS

c:\Users\mohaa\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:


# Configuration du modèle d'embeddings (doit être le même que pour la création de l'index)
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Chargement de la base vectorielle locale
# allow_dangerous_deserialization est nécessaire pour charger des fichiers .pkl localement
vector_db = FAISS.load_local(
    "faiss_index_pfe", 
    embeddings, 
    allow_dangerous_deserialization=True
)

print("Index FAISS chargé avec succès.")

Index FAISS chargé avec succès.


In [4]:
# Configuration du LLM Mistral
# temperature=0 garantit des réponses factuelles et non créatives
llm = ChatOllama(
    model="mistral", 
    temperature=0,
    num_gpu=99,  # Tente d'utiliser le GPU au maximum
    num_thread=4
)

print("Modèle Mistral prêt.")

Modèle Mistral prêt.


In [ ]:
# 4. Fonction de recherche + réponse modifiée
def ask_question(query):
    tracker = EmissionsTracker(save_to_file=True, output_dir=".")
    tracker.start()
    
    start_time = time.time() # Début du chrono
    # Recherche des documents pertinents (on récupère les objets 'Document' entiers)
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'fetch_k': 20})
    docs = retriever.invoke(query)
    # Préparation du contexte texte
    context_chunks = []
    #print("\n=== DOCUMENTS AND CONTEXT CHUNKS ===")
    for i, doc in enumerate(docs):
        # Découpe le texte en chunks
        text = doc.page_content
        context_chunks.append(text)
    # Combine les chunks pour le LLM
    context_for_llm = "\n\n".join(context_chunks)
# Prompt système optimisée pour un rôle de Tuteur CNN
    system_prompt = (
    "ROLE: CNN Specialist Assistant - STRICT DATA RETRIEVAL ONLY.\n\n"
    
    "=== THE GOLDEN RULE ===\n"
    "You are an AI assistant specialized ONLY in Convolutional Neural Networks (CNN). "
    "Your knowledge is strictly limited to the provided documentation (RAG).\n\n"
    
    "=== STRICT BEHAVIORAL PROTOCOL ===\n"
    "If the answer is not explicitly found within the provided context, or if the question is outside the scope of Convolutional Neural Networks (CNN), you must respond EXACTLY and ONLY with: 'I don't know.' Do not provide any other text."
    
    "2. 100% GROUNDING: Do not use your own training data. You are forbidden from "
    "generating information that is not explicitly present in the provided context. "
    "If the answer is missing from the context: 'I don't know.'\n"
    
    "3. 100% CONTEXT: You must answer based ONLY on the documentation. Do not add outside "
    "definitions of Quantum Computing. If the answer is not a CNN technical detail found in the text, "
    "the response is: 'I don't know.'\n"
    
    "4. NO FORMATTING/PREAMBLE: Do not use greetings (Hello, Hi). Start the answer "
    "directly with the relevant technical content or the refusal string.\n\n"
    
    "=== VERIFICATION ===\n"
    "Question is CNN + In Context -> Direct answer.\n"
    "Question is NOT CNN OR Not In Context -> I don't know."
    )
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=(
            f"STUDY CONTEXT:\n{context_for_llm}\n\n"
            f"QUESTION DE L'ÉTUDIANT : {query}\n\n"
            "RÉPONSE DU TUTEUR :"
        ))
    ]
    
    # Génération de la réponse
    response = llm.invoke(messages)

    end_time = time.time() # Fin du chrono
    emissions = tracker.stop() # Fin du suivi carbone

    inference_time = end_time - start_time
    # On retourne la réponse ET les documents sources
    return response.content , inference_time, emissions

In [6]:
# Question à poser
query = "What is maxpooling definition" 

print(f"Question: {query}")
print("-" * 30)

# Appel de la fonction
answer, duration, co2 = ask_question(query)

# Affichage des résultats
print("\n=== METRIQUES DE PERFORMANCE ===")
print(f"⏱️ Temps d'inférence : {duration:.2f} secondes")
print(f"🌱 Émissions générées : {co2:.10f} kg CO2")
print("-" * 30)
print("\n=== TUTOR RESPONSE ===")
print(answer)

[codecarbon WARNING @ 18:42:08] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:42:08] [setup] RAM Tracking...
[codecarbon INFO @ 18:42:08] [setup] CPU Tracking...


Question: What is maxpooling definition
------------------------------


[codecarbon WARNING @ 18:42:10] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 18:42:10] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz
[codecarbon WARNING @ 18:42:10] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:42:10] [setup] GPU Tracking...
[codecarbon INFO @ 18:42:10] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:42:10] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 18:42:10] >>> Tracker's metadata:
[codecarbon INFO @ 18:42:10]   Platform system: Windows-11-10.0.26200-SP0
[codecarbon INFO @ 18:42:10]   Python version: 3.14.0
[codecarbon INFO @ 18:42:10]   CodeCarbon version: 3.2.1
[codecarbon IN

POOLING LAYERS

Q1: What is pooling and why use it?
A: Pooling reduces spatial dimensions to:
- Decrease number of parameters and computations
- Create translation invariance
- Expand receptive field
- Reduce overfitting

Q2: What are the different types of pooling?
A:
import torch.nn as nn

# Max Pooling (most common)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

# Average Pooling
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

# Global Average Pooling
global_avgpool = nn.AdaptiveAvgPool2d((1, 1))

# Global Max Pooling
global_maxpool = nn.AdaptiveMaxPool2d((1, 1))

Q3: Difference between Max and Average Pooling?
A:
- Max Pooling: Takes maximum value in region
  → Better for detecting features (presence/absence)
  → More robust to noise
  
- Average Pooling: Computes average of region
  → Smooths features
  → Often used in final layer (Global Average Pooling)

processed concurrently. If not set, the tf.data runtime decides what it
should be based on available CPU. If num_parallel_c

[codecarbon INFO @ 18:42:29] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:42:29] Delta energy consumed for CPU with constant : 0.000751 kWh, power : 180.0 W
[codecarbon INFO @ 18:42:29] Energy consumed for All CPU : 0.000751 kWh
[codecarbon INFO @ 18:42:29] Energy consumed for all GPUs : 0.000034 kWh. Total GPU Power : 8.062421632643536 W
[codecarbon INFO @ 18:42:29] 0.000827 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 18:42:44] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:42:44] Delta energy consumed for CPU with constant : 0.000750 kWh, power : 180.0 W
[codecarbon INFO @ 18:42:44] Energy consumed for All CPU : 0.001501 kWh
[codecarbon INFO @ 18:42:44] Energy consumed for all GPUs : 0.000044 kWh. Total GPU Power : 2.397759444786034 W
[codecarbon INFO @ 18:42:44] 0.001628 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 18:


=== METRIQUES DE PERFORMANCE ===
⏱️ Temps d'inférence : 86.15 secondes
🌱 Émissions générées : 0.0002727148 kg CO2
------------------------------

=== TUTOR RESPONSE ===
 MaxPooling is a pooling operation that reduces spatial dimensions by taking the maximum value in a region (downsampling).
